<a href="https://colab.research.google.com/github/mountaha-ghabri/spark-RDD/blob/main/spark_core_api_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>


<hr color="black">

<center><h1>1. Introduction to the Spark Core API</h1></center>

<hr color="black">


Spark is a distributed computing framework designed to be fast and general-purpose.
It is used to develop applications that can execute **parallel computations** on data distributed across servers in a **cluster**.

Spark provides several APIs (Application Programming Interfaces) for the implementation of parallel computations.

This tutorial introduces the low-level **Spark Core API**.


---

__Documentation.__ For further information, please refer to the [official documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html).

---


**Execute the following cell to install the necessary tools to run Spark code.**

⏰ The cell will run for about 1 minute and 30 seconds.

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

**Execute the following cell to initialize the SparkContext and download some datasets.**

In [ ]:
import pyspark
import random
sc = pyspark.SparkContext(appName="code-tutorial-1")
print("Initialization successful")

!wget -q https://gquercini.github.io/courses/plp/tutorials/data.tgz
!tar xf data.tgz


# Resilient Distributed Datasets (RDDs)

Data distributed across servers in a cluster is represented by the Spark Core API through a data structure called Resilient Distributed Dataset (RDD).

A **Resilient Distributed Dataset (RDD)** is an immutable, distributed collection of items (e.g., integer, strings, objects).

The items of an RDD are organized into distinct **partitions**.
Each partition is stored independently on a server of the cluster;
two distinct partitions may reside on the same server.



Two ways exist to create a RDD:

* from a collection of elements in **main memory**.

* from an **external data source**, such as a file, a collection of files or a database.

## RDDs from in-memory collections

Lists, sets, dictionaries are example of collections whose items reside in main memory.

We use the function ``parallelize()`` to create an RDD from an in-memory collection.

In [ ]:
# A Python list.
men_list = ['mark', 'anthony', 'nick', 'charles', 'seth', 'joe', 'matthew', 'michael', 'andrea']

# Create a RDD from the list
men_rdd = sc.parallelize(men_list)

We can get the number of partitions of the new RDD with the function ``getNumPartitions()`` and the content of each partition with the function ``glom()``.

👉 Knowing the content of the partitions is not necessary to implement Spark computations. This is only presented here to show what is under the hood.

In [ ]:
print(f"Number of partitions of men_rdd: {men_rdd.getNumPartitions()}")

# We'll explain the function collect later.
for i, partition in enumerate(men_rdd.glom().collect()):
  print(f"Content of partition {i}: {partition}")

---

🤔 **How is the number of partitions determined?**

* When we run Spark on a single machine (as is our case), the number of partitions equals the **number of CPU cores** of the local machine. If the local machine has only one core, Spark creates 2 partitions.

* When we run Spark on a cluster, and we call ``parallelize()`` on a collection, the contents of the collection reside in the main memory of the server where ``parallelize()`` is called. If the collection is large, Spark splits the collection up into as many partitions as the **number of CPU cores across all servers** in the cluster. Then it distributes the partitions across these servers.
For small collections, Spark only creates two partitions and keeps them on a single server.

---

👉 Here our code is **run in a virtual machine** on Google Colab.

**The code is not run on your local machine.**

💡 Do you want to know **how many cores** the virtual machine has? **Execute the following command.**






In [ ]:
!nproc

2


## RDDs from external data sources

Spark can create RDDs from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra (a NoSQL database), HBase (the Hadoop database), Amazon S3 (a Cloud storage service). Spark supports text files, sequence files (essentially, binary files containing key/value pairs) and any Hadoop input format.


Different functions exist to create an RDD from an external source (``textFile()``, ``sequenceFile()``, ``pickleFile()``...).

The function ``textFile()`` is used to create a RDD from the content of a text file.

👉 Each item of the new RDD is a line of the file.



In [ ]:
data_file = "./data/moby-dick.txt"
lines_rdd = sc.textFile(data_file)

# The function take() is an action that shows the first 20 items of the RDD.
# We'll study actions in the next sections.
lines_rdd.take(20)

---

🤔 **How is the number of partitions determined?**

* When we run Spark on a single machine (as is our case), the number of partitions equals the **number of CPU cores** of the local machine. If the local machine has only one core, Spark creates 2 partitions.

* When we run Spark on a cluster, and we open a file stored in a distributed file system, such as HDFS, Spark creates an RDD with as many partitions as the **number of blocks** of the file. Note that these blocks are already distributed across the servers of the cluster.

---

👉 Two types of functions can be called on a RDD: **transformations** and **actions**.

# Transformations

A **transformation** is a function that takes in one or more RDDs and returns a new RDD.

A transformation applies a computation on the input RDD. The execution of a transformation triggers the creation of as many computational tasks as the number of partitions in the RDD. Each task applies the computation on a specific partition. Since partitions are disjoint, these tasks can run in parallel.


Let's create an RDD and study common transformations.

In [ ]:
# A Python list.
men_list = ['mark', 'anthony', 'nick', 'charles', 'seth', 'joe', 'matthew', 'michael', 'andrea']

# Create a RDD from the list
men_rdd = sc.parallelize(men_list)

## Transformation ``map()``

The transformation ``map()`` takes in an RDD $R$ and a function $f$ and returns a new RDD by applying $f$ to each element of $R$.

__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* A function $f$.

__Output.__

* A __new RDD__ $R^\prime = [f(r_i) | r_i \in R]$.


In the following example, the function $f$ is specified as an anonymous function (a *lambda function*): the function takes in a single item of the input RDD and returns a value.

In [ ]:
# collect() is an action that returns a list from the given RDD.
print(f"Content of the input RDD: {men_rdd.collect()}")

men_rdd_map = men_rdd.map(lambda name: name.capitalize())

print(f"Content of the output RDD: {men_rdd_map.collect()}")

We can also specify $f$ as a regular Python function and pass it to ``map()`` as an argument.

In [ ]:
def capitalize_name(name):
  return name.capitalize()

# collect() is an action that returns a list from the given RDD.
print(f"Content of the input RDD: {men_rdd.collect()}")

men_rdd_map = men_rdd.map(capitalize_name)

print(f"Content of the output RDD: {men_rdd_map.collect()}")

The following example uses the RDD ``lines_rdd`` that we created above.

👉 Each item of ``lines_rdd`` is a line from a file.

👉 The transformation ``map()`` applies the function ``split()`` to each item of ``lines_rdd``.

👉 An item of the RDD returned by the transformation ``map()`` is the list of words occurring in a line.


In [ ]:
print(f"Content of the input RDD: {lines_rdd.take(8)}")

lines_rdd_map = lines_rdd.map(lambda line: line.split())

print(f"Content of the output RDD: {lines_rdd_map.take(8)}")

## Transformation ``flatMap()``

The transformation ``flatMap()`` does the same thing as the transformation ``map()``, with the following difference. If each element of the RDD obtained from applying ``map()`` is a collection (a list, a set, or a tuple), the application of ``flatMap()`` "flattens" the collection.



Consider the last example again.
The Python function ``split()`` splits a given string into its constituent words; it returns a list.

In [ ]:
print(f"Content of the input RDD: {lines_rdd.take(8)}")

lines_rdd_map = lines_rdd.map(lambda line: line.split())

print(f"Content of the output RDD (after map): {lines_rdd_map.take(8)}")

However, if we want each item of  the output RDD to be a single word (in other words, we want to flatten each list), we can use ``flatMap()``.

In [ ]:
lines_rdd_flatmap = lines_rdd.flatMap(lambda line: line.split())
print(f"Content of the output RDD (after flatMap): {lines_rdd_flatmap.take(10)}")

## Transformation ``filter()``

The transformation ``filter()`` takes in an RDD $R$ and a predicate $p$ and returns a new RDD containing only the elements of $R$ that satisfy $p$.

👉 A **predicate** is a function that returns a Boolean value.

__Input.__  

* A RDD $r = [r_i | 0\leq i \leq k-1]$.
* A predicate $p$.

__Output.__

* A __new RDD__ $R^\prime = [r_i | r_i \in R \land p(r_i)]$.

In [ ]:
print(f"Content of the input RDD: {men_rdd_map.collect()}")

men_rdd_filter = men_rdd_map.filter(lambda name: name[0] =="M")

print(f"Content of the output RDD (after filter): {men_rdd_filter.collect()}")

## Transformation ``union()``

The transformation ``union()`` takes in two RDDs and returns a new RDD representing the union of the two input RDDs.





__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* A RDD $S = [s_i | 0\leq i \leq m-1]$.

__Output.__

* A __new RDD__ $U = [u_i | u_i \in R \lor u_i \in S]$.

👉 Unlike for the corresponding mathematical operation, the result of the transformation ``union()`` may contain duplicate values (e.g., Andrea in our example).

In [ ]:
women_rdd = sc.parallelize(['Anna', 'Laura', 'Emma', 'Charlotte', 'Sophia', 'Isabella', 'Mia', 'Andrea'])

print(f"Content of the first input RDD: {men_rdd_map.collect()}")

print(f"Content of the second input RDD: {women_rdd.collect()}")

people_rdd = women_rdd.union(men_rdd_map)

print(f"Content of the output RDD (after union): {people_rdd.collect()}")


## Transformation ``distinct()``

The transformation ``distinct()`` takes in an RDD and returns a new RDD containing the distinct elements in the input RDD.

__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.

__Output.__

* A __new RDD__ $R^\prime = [r_i | r_i \in R \land (r_i = r_j \implies i = j)]$.

👉 Note that the items in the output RDD are not in the same order as in the input RDD.
This is due to the fact that the transformation ``distinct()`` shuffles the elements so that duplicates will be in the same partition.

In [ ]:
print(f"Content of the input RDD: {people_rdd.collect()}")

people_rdd_distinct = people_rdd.distinct()

print(f"Content of the output RDD (after distinct): {people_rdd_distinct.collect()}")


## Transformation ``intersection()``

The transformation ``intersection()`` takes in two RDDs and returns a new RDD that contains the items that are common to both RDDs.

__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* A RDD $S = [s_i | 0\leq i \leq m-1]$.

__Output.__

* A __new RDD__ $T = [t_i | t_i \in R \land t_i \in S \land (t_i = t_j \implies i = j)]$.


👉 The output does not contain any duplicate elements, even if the input RDDs did.


In [ ]:
print(f"Content of the first input RDD: {people_rdd.collect()}")

print(f"Content of the second input RDD: {women_rdd.collect()}")

intersect_rdd = people_rdd.intersection(women_rdd)

print(f"Content of the output RDD (after intersect): {intersect_rdd.collect()}")



## Transformation ``subtract()``

The transformation ``subtract()`` takes in two RDDs and returns a new RDD containing the items of the first RDD that are not in the second RDD.


__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* A RDD $S = [s_i | 0\leq i \leq m-1]$.

__Output.__

* A __new RDD__ $T = [t_i | t_i \in R \land t_i \notin S]$.


In [ ]:
print(f"Content of the first input RDD: {people_rdd.collect()}")

print(f"Content of the second input RDD: {men_rdd_map.collect()}")

subtract_rdd = people_rdd.subtract(men_rdd_map)

print(f"Content of the output RDD: {subtract_rdd.collect()}")


## Transformation ``cartesian()``

The transformation ``cartesian()`` takes in two RDDs and returns a new RDD that is the Cartesian product of the two input RDDs.

__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* A RDD $S = [s_i | 0\leq i \leq m-1]$.

__Output.__

* A __new RDD__ $T = [(r_i, s_i) | \forall r_i \in R\ \forall s_i \in S]$.

In [ ]:
print(f"Content of the first input RDD: {men_rdd_map.collect()}")

print(f"Content of the second input RDD: {women_rdd.collect()}")

cartesian_rdd = men_rdd_map.cartesian(women_rdd)

print(f"Content of the output RDD: {cartesian_rdd.collect()}")


# Actions

An **action** is a function that takes in a RDD and returns a value, other than a RDD.

An action is applied to each partition of the input RDD. The value obtained from each partition is sent to the driver of the Spark application.

👉 As a result, an action always generates some traffic on the network, as the result obtained from each partition is sent back to the server where the driver is running.

## Action ``count()``

The action ``count()`` takes in a RDD and returns the number of elements in the RDD.



__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.

__Output.__

* The number $k$ of items in $R$.


In [ ]:
people_rdd.count()

## Action ``first()``

The action ``first()`` takes in an RDD and returns the first element of the RDD.




__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.

__Output.__

* $r[0]$, the first item in $R$.



In [ ]:
people_rdd.first()

## Action ``take()``

The action ``take()`` takes in an RDD and an integer $n$, and returns the first
$n$ items of the RDD.





__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* An integer $n$.

__Output.__

* The first $n$ items from $R$.


👉 It works by first scanning one partition, and uses the results from that partition to estimate the number of additional partitions needed to satisfy the limit.


In [ ]:
people_rdd.take(5)

👉 Similar functions are ``takeOrdered()`` (documented [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.takeOrdered.html)) and ``takeSample()`` (documented [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.takeSample.html)).

## Action ``collect()``

The action ``collect()`` takes in a RDD and returns a list containing all the elements of the RDD.




__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.

__Output.__

* A Python list with all the elements of $R$.

👉 All the elements of the RDD are sent to the driver. Depending on the size, this may cause a high network traffic and may result in the overflow of the memory of the server hosting the driver.
The use of ``collect()`` is discouraged for large RDDs.

In [ ]:
women_rdd.collect()

## Action ``reduce()``

The action ``reduce()`` takes in a RDD $R$ and a function $f$ and returns a value that is obtained by applying function $f$ to all elements of $R$ pairwise.



__Input.__  

* A RDD $R = [r_i | 0\leq i \leq k-1]$.
* A function $f$ that takes in two arguments.

👉 The two arguments of $f$ __must have the same type__ of the elements of $R$.

👉 Function $f$ must be **commutative** and **associative**.

__Output.__ The result is computed in the following way:

```
result = r_1
for i in range(2, k):
  result = f(r_i, result)
```



In the following example:

* we read the content of file ``./data/moby-dick.txt`` (function ``sc.textFile()``);

* we compute the number of characters in each line (function ``map(lambda x: len(x)``);

* we sum pairwise the number of characters in each line to obtain the total number of characters in the file (function ``reduce()``).

👉 Note how the transformations are chained together in lines 5-6. This programming style is common in Spark programs.


In [ ]:
data_file = "./data/moby-dick.txt"

nb_chars_per_line = sc.textFile(data_file)\
              .map(lambda x: len(x))

nb_chars = nb_chars_per_line.reduce(lambda x, y: x + y)
print(nb_chars)

NameError: name 'sc' is not defined

## Action ``saveAsTextFile()``

The action ``saveAsTextFile()`` takes in an RDD and a filename and stores the content of the RDD to the file.

In [ ]:
women_rdd.saveAsTextFile('./data/women_rdd.txt')

👉 If you look at your local file system, you'll see that the created file is actually a directory, that contains a file for each partition of the saved RDD.

👉 Similar functions exist to save the file to other formats.

# Pair RDDs

In pair RDDs each item is a pair $(k, v)$, where $k$ is referred to as a __key__ and $v$ as a __value__.

In [ ]:
pair_rdd = sc.parallelize([("cat", 2), ("owl", 3), ("dog", 5), ("cat", 2), ("dog", 1), ("cow", 1), ("cat", 3), ("owl", 4), ("tiger", 1)])
pair_rdd.collect()

Pair RDDs accept all the transformations and actions presented above.

They also accept transformations and actions that are specifically tailored to handle key-value pairs.

## Transformation ``reduceByKey()``

The transformation ``reduceByKey()`` takes in a pair RDD $R$ and a commutative and associative reduce function $f$, and returns a new pair RDD with the same keys as $R$; the values associated with each key are merged using $f$.

__Input.__  

* A RDD $R = [(k_i, v_i) | 0\leq i \leq n-1, i\neq j  \nRightarrow k_i \neq k_j]$.

* A function $f$ that takes two arguments.


👉 The two arguments of $f$ __must have the same type__ of the __values__ (<u>not the keys</u>) in $R$.

👉 $f$ must be **commutative** and **associative**.



__Output.__

For each key $k$ in $R$, it applies ``reduce()`` to all values associated to $k$.

👉 ``reduce()`` is an action, ``reduceByKey()`` is a transformation.



In [ ]:
print(f"Content of the input RDD: {pair_rdd.collect()}")
pair_rdd_rbk = pair_rdd.reduceByKey(lambda x, y : x + y)

print(f"Content of the output RDD (after reduceByKey): {pair_rdd_rbk.collect()}")

## Transformation ``mapValues()``

The transformation ``mapValues()`` takes in a pair RDD $R$ and a function $f$ and returns a new pair RDD with the same keys as $R$; the function $f$ is applied to each value.






__Input.__  

* A RDD $R = [(k_i, v_i) | 0\leq i \leq n-1, i\neq j  \nRightarrow k_i \neq k_j]$.

* A function $f$.

__Output.__

* A __new RDD__ $R^\prime = [(k_i, f(v_i)) | (k_i, v_i) \in R ]$.


In [ ]:
print(f"Content of the input RDD: {pair_rdd.collect()}")

pair_rdd_mv = pair_rdd.mapValues(lambda x: x+1)

print(f"Content of the output RDD (after mapValues): {pair_rdd_mv.collect()}")

## Transformation ``groupByKey()``

The transformation ``groupByKey()`` takes in a pair RDD $R$ and returns a new pair RDD with the same keys as $R$; the values of each key are grouped  into a single collection.



__Input.__  

* A RDD $R = [(k_i, v_i) | 0\leq i \leq n-1, i\neq j  \nRightarrow k_i \neq k_j]$.

__Output.__

* A __new RDD__ $R^\prime = [(k_i, l) | l = [v_j | (k_i, v_j) \in R ]]$.

👉 ``groupByKey()`` returns an iterable.


In [ ]:
print(f"Content of the input RDD: {pair_rdd.collect()}")

pair_rdd_gbk = pair_rdd.groupByKey()

print(f"Content of the output RDD (after groupByKey): {pair_rdd_gbk.collect()}")


We can still convert each iterable to a list by using the transformation ``mapValues()``.

In [ ]:
pair_rdd_gbk_list = pair_rdd_gbk.mapValues(lambda it: list(it))

print(f"Content of the output RDD (after mapValues): {pair_rdd_gbk_list.collect()}")

## Transformation ``join()``

The transformation ``join()`` takes in two pair RDDs and creates a new RDD where the values associated with the same key are joined together.



__Input.__  

* A RDD $R_1 = [(k^1_i, v^1_i) | 0\leq i \leq n-1, i\neq j  \nRightarrow k^1_i \neq k^1_j]$.

* A RDD $R_2 = [(k^2_i, v^2_i) | 0\leq i \leq m-1, i\neq j  \nRightarrow k^2_i \neq k^2_j]$.

__Output.__

* A __new RDD__ $R^\prime = [(j, (v^1, v^2)) | \forall\ (j, v^1)\in R_1 \ \forall\ (j, v^2) \in R_2]]$.

👉 If a key occurs in $R_1$ but does not occur in $R_2$ (and the other way round), that key is not represented in the output of ``join()``.


In [ ]:
pair_rdd_text = sc.parallelize([("cat", "meow"), ("dog", "woof"), ("dog", "arf"), ("cow", "moo")])

print(f"Content of the first RDD: {pair_rdd.collect()}")
print(f"Content of the second RDD: {pair_rdd_text.collect()}")

pair_rdd_join = pair_rdd.join(pair_rdd_text)

print(f"Content of the output RDD (after join): {pair_rdd_join.collect()}")



# Narrow and wide transformations

Some transformations apply a computation on an RDD without the need of moving data from one server to another server.
For example, the transformation ``map()`` applies a function $f$ to all items of an input RDD and does not need to move the items from one partition to another partition in doing so.
In other words, the contents of partition $i$ of the output RDD is determined by applying function $f$ on all items of partition $i$ of the input RDD. The operation is local to each single partition.

In general, a **narrow transformation** is one where each partition of the output RDD is computed from one single partition of the input RDD.
Examples of narrow transformations are: ``map``, ``flatMap``, ``filter`` and ``union``.

Other transformations need to move items from one partition to another partition (hence, from one server to another server).

👉 The action of moving data from one partition to another partition is called **shuffle**.

For example, ``reduceByKey()`` needs to move to the same partition the pairs sharing the same key.

In general, a **wide transformation** in one that triggers a data shuffle.
Examples of wide transformations are: ``distinct()``, ``intersection()``, ``join()``, ``reduceByKey()`` and ``groupByKey()``.


# Example: word count

The following code defines a function ``wordcount()`` that counts the number of occurrences of each word in a given file.
The comments should help you understand the code.

👉 Note how the transformations are chained. This is a common coding style in Spark programs.

In [ ]:
def wordcount(content_file, stopwords_file):

  '''
  Counts the number of occurrences of the words in a given file (first argument).
  The function ignores stop words (insignificant words, such as articles, prepositions)
  that are specified in a file (second argument).
  The returned words are sorted by decreasing frequency (frequent words come first).

  Parameters
  -----------
    content_file (str): path to the file with the text to analyze.
    stopwords_file (str): path to the file that contains the stopwords.


  Returns
  --------
    rdd.
      An RDD of pairs (w, ow), where w is a word and ow is the number of occurrences
      of the word in content_file.
  '''

  # Read the stopwords from the given file into a list.
  stopwords = []
  with open(stopwords_file, "r") as f:
    for line in f:
        stopwords.append(line.lower().strip())
  '''
  Here is the meaning of each transformation:

  textFile: Read the content of the file into a RDD.
  flatMap: Split each line into its constituent words.
  filter: Remove stop words.
  map: Associate the value 1 to each word
  reduceByKey: For each word, sum up all the 1s associated to that word.
  sortBy: Sort by decreasing frenquency and return the result.
  '''
  return sc.textFile(content_file)\
            .flatMap(lambda x: x.split())\
            .map(lambda x: x.lower())\
            .filter(lambda x: x not in stopwords)\
            .map(lambda x: (x, 1))\
            .reduceByKey(lambda x, y: x+y)\
            .sortBy(lambda x: x[1], ascending=False)

# Call the function. The function returns an RDD.
word_counts = wordcount('./data/moby-dick.txt', './data/stopwords.txt')

# Show the first ten items in the RDD returned by the function.
word_counts.take(10)


---

**Good to know.** In the previous program, calling the function ``wordcount()`` does not trigger the execution of the transformations. The transformations are only called when we call the first action on the RDD returned by ``wordcount()``. This execution mode is referred to as **lazy evaluation**.

👉 Always remember to call an action on an RDD is you want the transformations on that RDD to be executed.

---

# Summary

* Spark is a distributed computing framework used to develop applications that can execute **parallel computations** on data distributed across servers in a **cluster**.

* Parallel computations on distributed data are implemented as operations on data structures called **Resilient Distributed Datasets (RDDs)**.

* RDDs (Resilient Distributed Datasets) are **immutable**, **distributed** collections of items (e.g., integer, strings, objects).

* Items in an RDD are divided into **disjoint partitions**. A partition resides on a single server; two distinct partitions can reside on the same server.

* A Spark program creates a new **RDD**, either from an in-memory collection (for debugging or prototyping purposes) or from external sources (production environments).

* A Spark program manipulates RDDs through functions that are called **transformations**. A transformation takes in one or several RDDs and returns a new RDD. A tranformation never modifies the contents of the input RDDs. A transformation applies a computation on an input RDD through parallel computational tasks that operate on each partition of the RDD.

* **Wide** are costlier than **narrow** transformations, as they always trigger a data shuffle (move data from one partition to another).

* The result of a Spark program is obtained by calling an **action**, that is a function that takes in one RDD and returns a value.
Only when an action on a RDD is invoke are the transformations on that RDD actually executed (**lazy evaluation**).









# Advanced considerations

The way items are partitioned in a RDD may impact the performances of transformations and actions.

Let's illustrate this point with an example.



In the following cell, we force Spark to create four partitions of a new RDD by adding a second argument to the function ``parallelize()``.

In [ ]:
l1 = [("cat", 2), ("owl", 3), ("dog", 5), ("cat", 2), ("dog", 1), ("cow", 1), ("cat", 3), ("owl", 4), ("tiger", 1)]

rdd_l1 = sc.parallelize(l1, 4)

print(f"Number of partitions of rdd_l: {rdd_l1.getNumPartitions()}")

for i, partition in enumerate(rdd_l1.glom().collect()):
  print(f"Content of partition {i}: {partition}")


👉 Two items with the same key are not necessarily in the same partition.

Now, let's create another RDD, where items share some of the keys with the RDD created above.

In [ ]:
rdd_l2 = sc.parallelize([("cat", "meow"), ("dog", "woof"), ("dog", "arf"), ("cow", "moo")])

print(f"Number of partitions of rdd_l: {rdd_l2.getNumPartitions()}")

for i, partition in enumerate(rdd_l2.glom().collect()):
  print(f"Content of partition {i}: {partition}")


The following code joins the two RDDs.

In [ ]:
rdd_join = rdd_l1.join(rdd_l2)

print(f"Number of partitions of rdd_join: {rdd_join.getNumPartitions()}")

for i, partition in enumerate(rdd_join.glom().collect()):
  print(f"Content of partition {i}: {partition}")


The following figure shows what happens when we join the two RDDs.

<center><img src="https://drive.google.com/uc?export=view&id=11H7wYqMf_SauPHk4-qoFmoPF8kmm7Xnq" width="800"  /></center>




Data of <u>both RDD</u> need to be shuffled. More precisely:

1. The pairs sharing the same key in the first RDD are moved to the same partition.

2. The pairs sharing the same key in the second RDD are moved to the same partition.

👉 The destination partition of a key-value pair is computed by applying a hash function on the key; therefore, if two pairs have the same key, they will land on the same output partition, regardless of whether they belong to the same input RDD or not.

3. Now that all pairs from both RDDs sharing the same key  are in the same partition, they can be joined.

👉 Two different partitions may reside on two different servers. Therefore, copying data from one partition to another leads to data being shuffled on the network. **This slows down the computation**.



If we know that the RDD ``rdd_l1`` will be joined many times with other RDDs, it is worth taking the time to **hash-partition** ``rdd_l1`` before calling the join transformations.

👉 A RDD is **hash-partitioned** when any two pairs sharing the same key are in the same partition (hence, they are stored on the same server).

The following code shows how to hash partition an RDD.

In [ ]:
# Define a new HashPartitioner
class HashPartitioner:
    def __init__(self, num_partitions):
        self.num_partitions = num_partitions

    def __call__(self, key):
        return hash(key) % self.num_partitions

rdd_l1_hash = rdd_l1.partitionBy(4, HashPartitioner(4))


print(f"Number of partitions of rdd_l1_hash: {rdd_l1_hash.getNumPartitions()}")

for i, partition in enumerate(rdd_l1_hash.glom().collect()):
  print(f"Content of partition {i}: {partition}")


Now, if we try to join ``rdd_l1`` with ``rdd_l2``, only the pairs of ``rdd_l2`` will be shuffled, which results is a lower execution time.


<center><img src="https://drive.google.com/uc?export=view&id=1a9emjO6h9Ld8TtgvDCY9ahKnwFpRKBi4" width="800"  /></center>



## About hash partitioning

It is not necessary to explicitly hash-partition an RDD if we want to join this RDD to another one only once.
In general, we should hash-partition an RDD when:

* the RDD is large ;

* we intend to join the RDD with other RDDs many times.




Note that any RDD obtained from joining two RDDs is **hash-partitioned**.

The trasformations that hash-partition their output RDDs are: ``cogroup()``, ``groupWith()``, ``join()``, ``leftOuterJoin()``, ``rightOuterJoin()``, ``groupByKey()``, ``reduceByKey()``, ``combineByKey()``, ``partitionBy()``, ``sort()``.

The following transformations: ``mapValues()``, ``flatMapValues()`` and ``filter()`` guarantee that the output is hash-partitioned if their input is.


## The case of ``map`` and ``mapValues``

Consider again the RDD ``rdd_l1``.



In [ ]:
rdd_l1.collect()

This RDD is not hash-partitioned. We can also confirm it by checking that the partitioner associated with the RDD is ``None``.

In [ ]:
print(f"Partitioner associated with the rdd_l1: {rdd_l1.partitioner}")

Now, let's apply a ``reduceByKey()`` transformation.

The RDD returned by the transformation ``reduceByKey()`` has a partitioner associated.

In [ ]:
rdd_l1_rbk = rdd_l1.reduceByKey(lambda x, y: x+y)

print(f"Partitioner associated with the rdd_l1_rbk: {rdd_l1_rbk.partitioner}")
print(f"Content of the rdd_l1_rbk: {rdd_l1_rbk.collect()}")

In the following cell, we apply to ``rdd_l1_rbk`` (the result of ``reduceByKey()``) a transformation ``map()`` that does not change the keys of the RDD.

Which partitioner is associated to the result of ``map()``?

In [ ]:
rdd_l1_rbk_map = rdd_l1_rbk.map(lambda x: (x[0], x[1] + 1))

print(f"Partitioner associated with the rdd_l1_rbk_map: {rdd_l1_rbk_map.partitioner}")
print(f"Content of the rdd_l1_rbk_map: {rdd_l1_rbk_map.collect()}")

Now, look at what happens when we use the transformation ``mapValues()`` instead of the transformation ``map()``.

In [ ]:
rdd_l1_rbk_mapvalues = rdd_l1_rbk.mapValues(lambda x: x+1)

print(f"Partitioner associated with the rdd_l1_rbk_mapvalues: {rdd_l1_rbk_mapvalues.partitioner}")
print(f"Content of the rdd_l1_rbk_mapvalues: {rdd_l1_rbk_mapvalues.collect()}")

---

**Good to know.** The ``map()`` transformation does not preserve the partitioner associated to the input RDD. In fact, we pass ``map()`` a function for which Spark has no way to know whether it modifies the keys or not.
As opposed to that, ``mapValues()`` is a transformation that never touches the keys of the input RDD, therefore, Spark can confidently preserve the partitioner associated with the input RDD.



---